# Reading keys

Create a file called `.env`
and in it put:

```
AZURE_OPENAI_API_KEY='...'
```

In [1]:
from dotenv import load_dotenv
from pprint import pprint
import getpass
import os

load_dotenv()

AZURE_OPENAI_API_KEY = os.environ['AZURE_OPENAI_API_KEY']
GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']

def debug(s, title=None):
    if title:
        print(f"[*] {title}")
    print(f"[*] {s}")
    

# IMPORTANT:
Set production flag to True to use OpenAI

In [2]:
PRODUCTION = True

In [3]:
from langchain.chat_models import AzureChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage, SystemMessage

## LLM Configuration

In [6]:
config = {
    "openai_azure": {
        "api_key": AZURE_OPENAI_API_KEY,
        "azure_deployment": "team1-gpt4o", # The deployment name
        "azure_endpoint": "https://096290-oai.openai.azure.com",
        "openai_api_version": "2023-05-15",
        "temperature": 0.7
    }
}

if PRODUCTION:
    # Init openai
    print("Using OpenAI production flag set to: ", PRODUCTION)
    chat = AzureChatOpenAI(**config["openai_azure"])
else:
    print("Using Gemini production flag set to: ", PRODUCTION)
    chat = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.9)

Using OpenAI production flag set to:  True


In [7]:
# Define a chat prompt template
prompt_template = ChatPromptTemplate.from_template("{input}")

# Example text completion function
def generate_completion(user_input):
    """
    Generates a completion using the Azure OpenAI Chat API via LangChain.
    Args:
    user_input (str): The input prompt from the user.
    Returns:
    str: The completion response from the model.
    """

    # Render the prompt
    formatted_prompt = prompt_template.format(input=user_input)
    messages = [HumanMessage(content=formatted_prompt)]

    # Generate response
    response = chat(messages=messages)
    return response.content

generate_completion("What is the capital of Belguim?")

'The capital of Belgium is **Brussels**.'

# JSON Structured Output

### Example

Good for short and not nested structures

In [13]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

response_schemas = [
    ResponseSchema(name="name", description="The name of the person"),
    ResponseSchema(name="age", description="The age of the person"),
    ResponseSchema(name="interests", description="A list of the person's interests")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

print(output_parser.get_format_instructions())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"name": string  // The name of the person
	"age": string  // The age of the person
	"interests": string  // A list of the person's interests
}
```


For more complex and nested structures used Pydantic

In [8]:
from pydantic import BaseModel, Field
from typing import List, Optional
from langchain_core.output_parsers import PydanticOutputParser

# Define Pydantic models for your structure
class Cravings(BaseModel):
    type: str = Field(description="The type of food craved (e.g., dessert, snack, meal)")
    style: str = Field(description="The style of food preferred (e.g., light, rich, savory)")
    flavor: str = Field(description="The flavor profile preferred (e.g., fruity, chocolatey, spicy)")

class UserProfile(BaseModel):
    age: int = Field(description="User's age in years")
    weight: float = Field(description="User's weight in kg")
    height: float = Field(description="User's height in cm")
    calorie_limit: float = Field(description="Maximum calories allowed per meal/snack")

class UserPreferences(BaseModel):
    cravings: Cravings
    available_ingredients: List[str] = Field(description="List of ingredients the user has available")
    user_profile: UserProfile
    
parser = PydanticOutputParser(pydantic_object=UserPreferences)

In [10]:
from langchain.prompts import PromptTemplate


def generate_struct(user_text, instructions, _debug=False):
    # Update the prompt template to guide the LLM to parse user text
    prompt_template = \
    """
    Extract and organize the information from the user's text into a structured format.
    {format_instructions}
    
    User Text: {user_text}
    
    Parse the above text to extract all relevant information about the user's cravings, available ingredients, and profile.
    """
    
    prompt = PromptTemplate(
        input_variables=["user_text"],
        partial_variables={"format_instructions": instructions},
        template=prompt_template
    )
    
    formatted_prompt = prompt.format(user_text=user_text)
    
    llm_output = chat.invoke(formatted_prompt)
    
    if not isinstance(llm_output, str):
        if _debug:
            debug("Output of LLM is not str") 
        llm_output = llm_output.text()
    
    if _debug:
        debug(formatted_prompt, "Formatted Output before invoking LLM")
        debug(llm_output, "LLM Output")
    
    try:
        return parser.parse(llm_output)
    except Exception as e:
        raise Exception(f"LLM may have returned unstructured output and cannot be parsed: {str(e)}")


# Example usage:
user_text = """
I am 28 years old, weigh about 65kg and I'm 170cm tall. 
I'm trying to watch my calories, so I want to stay under 300 calories for this snack.
I'm really craving something fruity and light for dessert.
I have some strawberries and yogurt in my fridge that I could use.
"""

instructions = parser.get_format_instructions()

out = generate_struct(user_text, instructions, _debug=True)


[*] Output of LLM is not str
[*] Formatted Output before invoking LLM
[*] 
    Extract and organize the information from the user's text into a structured format.
    The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"Cravings": {"properties": {"type": {"description": "The type of food craved (e.g., dessert, snack, meal)", "title": "Type", "type": "string"}, "style": {"description": "The style of food preferred (e.g., light, rich, savory)", "title": "Style", "type": "string"}, "flavor": {"description": "The flavor profile preferred (e.g., fruity, chocolatey, spicy)", "title": "Flav

In [18]:
import json
example_output = {
    "cravings": {
        "type": "desert",
        "style": "light",
        "flavor": "fruity"
    },
    "available_ingredients": ["strawberries", "yogurt"],
    "user_profile": {
        "age": 28,
        "weight": 65,
        "height": 170,
        "calorie_limit": 300
    }
}
pprint(out.dict())

{'available_ingredients': ['strawberries', 'yogurt'],
 'cravings': {'flavor': 'fruity', 'style': 'light', 'type': 'dessert'},
 'user_profile': {'age': 28,
                  'calorie_limit': 300.0,
                  'height': 170.0,
                  'weight': 65.0}}


/var/folders/9f/_6775r3x22z0p4kb6_fn2xmr0000gn/T/ipykernel_27249/2954805604.py:16: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  pprint(out.dict())
